In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!tar -zxvf /kaggle/input/200-bird-species-with-11788-images/CUB_200_2011.tgz

In [4]:
import tensorflow as tf
import numpy as np

In [5]:
import glob

In [6]:
img_path = glob.glob("./CUB_200_2011/images/*/*.jpg")

In [7]:
len(img_path)

In [8]:
img_path[1000]

In [9]:
img_path[1000].split("/")[3].split(".")[1]

In [10]:
all_labels = [img_p.split('/')[3].split(".")[1] for img_p in img_path]

In [11]:
all_labels[:4]

In [12]:
label_names = np.unique(all_labels)
# len(label_names)
label_names_enu = enumerate(label_names)
label_to_index = dict((i, k) for (k, i) in label_names_enu)
index_to_label = dict((k, i) for (i, k) in label_to_index.items())

all_labels = [label_to_index.get(name) for name in all_labels]


In [13]:
np.random.seed(2022)
random_index = np.random.permutation(len(img_path))

imgs_path = np.array(img_path)[random_index]
all_labels = np.array(all_labels)[random_index]

i = int(len(img_path)*0.2)
train_path = img_path[i:]
train_label = all_labels[i:]
test_path = img_path[:i]
test_label = all_labels[:i]


In [14]:
test_path

In [15]:

train_ds = tf.data.Dataset.from_tensor_slices((train_path, train_label))
test_ds = tf.data.Dataset.from_tensor_slices((test_path, test_label))


In [16]:
def load_img(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image,(256, 256))
    image = tf.cast(image, tf.float32)
    image = image / 255
    return image, label


In [17]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(load_img,num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(load_img,num_parallel_calls=AUTOTUNE)

BATCH_SIZE = 32
train_ds = train_ds.repeat().shuffle(300).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)


In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3), input_shape=(256,256,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(1024, activation="relu"))
model.add(tf.keras.layers.Dense(200))

model.summary()

In [19]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["acc"]
        )


In [20]:
STEPS_PER_EPOCH = int(len(train_path)//BATCH_SIZE)
VAL_STEPS_PER_EPOCH = int(len(test_path)/BATCH_SIZE)
history = model.fit(
        train_ds,epochs=10,
        batch_size=BATCH_SIZE,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=test_ds,
        validation_steps=VAL_STEPS_PER_EPOCH
        )

In [ ]:
import tensorflow as tf

In [ ]:
tf.test.is_gpu_available()

In [21]:
train_ds.take(1)

In [22]:
for i , k in train_ds.take(1):
    print(k)

In [23]:
for i , k in train_ds.take(1):
    print(i)